# Read grid function

In [1]:
import pandas as pd
import xarray as xr

In [2]:
hgrid = '/Users/brey/GLOBAL/SCHISM_GLOBAL_2/hgrid.gr3'

In [59]:
hgrid = '/Users/brey/SCHISM/test/hgrid.gr3'

In [3]:
#extract number of elements, number of nodes
ne,nn = pd.read_csv(hgrid,header=0,skiprows=1,nrows=0,delim_whitespace=True)
    
ne=int(ne)
nn=int(nn)        

ne, nn

(328298, 167796)

In [4]:
#read lon,lat,depth for all nodes
q=pd.read_csv(hgrid,skiprows=2,header=None,delim_whitespace=True,engine='python',nrows=nn,names=['id','x','y','z'])        
q=q.set_index(['id'])
q.head()    



,x,y,z
id,,,
1,-172.295365,89.965970,4121
2,0.025995,0.147022,4933
3,90.015117,0.072420,4197
4,-179.882480,0.029396,5262
5,-90.017139,0.077480,940


In [5]:
#read connectivity
e = pd.read_csv(hgrid,skiprows=nn+2,header=None,delim_whitespace=True,engine='python',nrows=ne,names=['id','nv','a','b','c'])
e=e.set_index(['id'])
e.head()    



,nv,a,b,c
id,,,,
1,3,57565,57567,57566
2,3,30277,62909,30276
3,3,152565,152895,131736
4,3,22773,109352,63330
5,3,49246,137011,130328


In [106]:
#Number of open boundaries
nob = pd.read_csv(hgrid,header=None,skiprows=ne+nn+2,nrows=1,delimiter='=')[0]
nob

0    0
Name: 0, dtype: int64

In [107]:
if nob.values > 0:
    #Total number of open boundary nodes
    ton = pd.read_csv(hgrid,header=None,skiprows=ne+nn+3,nrows=1,delimiter='=')[0]

    ns = ne + nn + 2 + 2
    nnob = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].values[0]
    label = ' '.join(pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:])
    op = pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnob,names=[label])
    for i in range(1, nob):
        ns = ns + 1 + nnob
        nnob = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].values[0]
        label = ' '.join( pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:] + [str(i + 1)] )
        op = pd.concat([op,pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnob,names=[label])], axis=1)

    op.index.name = 'id' # set index name to match q,e
    op.index = op.index + 1 #  ''
else:
    ns = ne + nn + 3
    nnob = 0
    op = pd.DataFrame({})

In [108]:
op.head()

""


In [109]:
#Number of land boundaries
nlb = pd.read_csv(hgrid,header=None,skiprows=ns + 1 + nnob,nrows=1,delimiter='=')[0]
nlb

0    373
Name: 0, dtype: int64

In [110]:
if nlb.values > 0:
    #Total number of land boundary nodes
    tln = pd.read_csv(hgrid,header=None,skiprows=ns + 1 + nnob + 1,nrows=1,delimiter='=')[0]
     
    ns = ns + 1 + nnob + 2
    nnlb, btype = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].str.split()[0]
    nnlb = int(nnlb)
    label = ' '.join(pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:])
    bt=[btype]
    lp = pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnlb,names=[label])
    for i in range(1, nlb):
        ns = ns + 1 + nnlb
        nnlb, btype  = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].str.split()[0]
        nnlb = int(nnlb)

        bt.append(btype)
        label = ' '.join( pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:-1] + [str(i + 1)] )
        lp = pd.concat([lp,pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnlb,names=[label])], axis=1)

    lp.index.name = 'id' # set index name to match q,e
    lp.index = lp.index + 1 #  ''
        

    #DataFrame with the type of land boundary
    bbt = pd.DataFrame({'type':bt})
    bbt.index.name = 'id'
    bbt.index = bbt.index + 1
else:
    lp=pd.DataFrame({})
    bbt = pd.DataFrame({})

land boundary 1


In [112]:
lp.head()

,land boundary 1,land boundary 2,land boundary 3,land boundary 4,land boundary 5,land boundary 6,land boundary 7,land boundary 8,land boundary 9,land boundary 10,...,land boundary 364,land boundary 365,land boundary 366,land boundary 367,land boundary 368,land boundary 369,land boundary 370,land boundary 371,land boundary 372,land boundary 373
id,,,,,,,,,,,,,,,,,,,,,
1,5705.0,16858.0,51533.0,3728.0,23,69200.0,72583.0,76144.0,18238.0,66320.0,...,60774.0,7786.0,34096.0,66278.0,66277.0,26528.0,43571.0,26522.0,90947.0,3729.0
2,5706.0,16859.0,52668.0,3730.0,51,87849.0,72584.0,108104.0,19092.0,66322.0,...,63557.0,7787.0,38225.0,68167.0,66278.0,26529.0,48736.0,26523.0,96506.0,3731.0
3,5708.0,16873.0,68478.0,75813.0,61,117821.0,72586.0,133899.0,20354.0,78206.0,...,75700.0,7796.0,44980.0,128846.0,68167.0,26531.0,68426.0,26528.0,110051.0,3733.0
4,5710.0,20281.0,69200.0,90154.0,62,119934.0,73472.0,136486.0,20355.0,135570.0,...,77724.0,7797.0,45442.0,128847.0,68168.0,26532.0,68427.0,26529.0,120482.0,90946.0
5,5713.0,20552.0,79268.0,90156.0,63,119935.0,81688.0,165399.0,123121.0,135572.0,...,84445.0,7804.0,45443.0,162447.0,82919.0,26534.0,120596.0,28475.0,162741.0,110051.0


In [113]:
# merge to one xarray DataSet
g = xr.merge([q.to_xarray(), e.to_xarray(), op.to_xarray(), lp.to_xarray(), bbt.to_xarray()])


In [114]:
g

<xarray.Dataset>
Dimensions:            (id: 328298, index: 0)
Coordinates:
  * id                 (id) int64 1 2 3 4 5 ... 328295 328296 328297 328298
  * index              (index) object 
Data variables:
    x                  (id) float64 -172.3 0.02599 90.02 -179.9 ... nan nan nan
    y                  (id) float64 89.97 0.147 0.07242 0.0294 ... nan nan nan
    z                  (id) float64 4.121e+03 4.933e+03 4.197e+03 ... nan nan
    nv                 (id) int64 3 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3
    a                  (id) int64 57565 30277 152565 ... 33612 167796 33612
    b                  (id) int64 57567 62909 152895 ... 167796 165674 165674
    c                  (id) int64 57566 30276 131736 ... 153642 87057 167796
    land boundary 1    (id) float64 5.705e+03 5.706e+03 5.708e+03 ... nan nan
    land boundary 2    (id) float64 1.686e+04 1.686e+04 1.687e+04 ... nan nan
    land boundary 3    (id) float64 5.153e+04 5.267e+04 6.848e+04 ... nan nan
    land bo